In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from skimage import transform

from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from triplet_loss import *

import glob
import cv2
import scipy
import pandas as pd
# os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3'

In [2]:
# import scipy
NCP_masks = glob.glob('/data2/Cpeng/NCP_lung_masks/*')
mask = cv2.imread(NCP_masks[5],0)
mask[mask>0]=1
print(mask.min())
print(mask.max())
print(mask.shape)
plt.imshow(mask)
mt = scipy.ndimage.distance_transform_edt(mask)
plt.figure()
plt.imshow(mt)

In [3]:
NCP_images = glob.glob('/data2/Cpeng/NCP_raw_images/*')
CP_images = glob.glob('/data2/Cpeng/CP_raw_images/*')
Normal_images = glob.glob('/data2/Cpeng/Normal_raw_images/*')


In [4]:
# lesion_slices = pd.read_csv('/data2/Cpeng/COVID/lesions_slices.csv')
# Normal_slices = pd.read_csv('/data2/Cpeng/COVID/Normal_slices.csv')
# print(len(lesion_slices))
# # print(len(Normal_slices))
# NCP_slices = lesion_slices['imgpath'].tolist()[36894:58766]
# CP_slices = lesion_slices['imgpath'].tolist()[0:36894]
# N_slices = Normal_slices['imgpath'].tolist()
# print(len(NCP_slices))
# print(len(CP_slices))
# print(len(N_slices))
# print(N_slices[0])

from sklearn.model_selection import train_test_split
NCP_train, NCP_val, _, _ = train_test_split(NCP_images, NCP_images, test_size=0.3, random_state=42)
CP_train, CP_val, _, _ = train_test_split(CP_images, CP_images, test_size=0.3, random_state=42)
Normal_train, Normal_val, _, _ = train_test_split(Normal_images, Normal_images, test_size=0.2, random_state=42)
# print(len(NCP_train))
# print(len(CP_train))
# print(len(Normal_train))
CP_Normal_train = []
CP_Normal_train.extend(CP_train)
CP_Normal_train.extend(Normal_train)                                        
CP_Normal_val = []
CP_Normal_val.extend(CP_val[:])
CP_Normal_val.extend(Normal_val[:])
# print(len(CP_Normal_train))
# print(len(CP_Normal_val))
# print(CP_Normal_val)

In [5]:
# Normal = glob.glob('/data2/Cpeng/COVID/Normal/*/*/*')
# print(Normal[0])
# Normal_df = pd.DataFrame({'imgpath':Normal})
# Normal_df.to_csv('/data2/Cpeng/COVID/Normal_slices.csv',index=False,sep=',')

In [6]:
from numpy import fliplr
class COVID_19(Dataset):
    def __init__(self, file_path_NCP, file_path_CP_Normal, phase='train'):
        self.path_list_NCP = file_path_NCP
        self.path_list_CP_Normal = file_path_CP_Normal
        self.phase = phase
        
    def __getitem__(self, index):
        
        num_NCP = len(self.path_list_NCP)
        num_CP_Normal = len(self.path_list_CP_Normal)
        
        image_anchor_NCP,mask_anchor_NCP= self.load_data(self.path_list_NCP[index])
        randn_aug_0=np.random.randint(0,3)
        if randn_aug_0==0:
            image_anchor_NCP = fliplr(image_anchor_NCP)
            mask_anchor_NCP = fliplr(mask_anchor_NCP)
        if randn_aug_0==1:
            image_anchor_NCP = transform.rotate(image_anchor_NCP,2)
            mask_anchor_NCP = transform.rotate(mask_anchor_NCP,2)
        if randn_aug_0==2:
            image_anchor_NCP = transform.rotate(image_anchor_NCP,-2)
            mask_anchor_NCP = transform.rotate(mask_anchor_NCP,-2)
        image_anchor_NCP,mask_anchor_NCP = self.normalize_data(image_anchor_NCP,mask_anchor_NCP)
        anchor_map = scipy.ndimage.distance_transform_edt(mask_anchor_NCP)
        if anchor_map.max()>0:
            anchor_map = anchor_map/anchor_map.max()
        ########## For Classification
        ### positive
        randn = np.random.randint(0,num_NCP)
        image_positive_NCP_D,mask_positive_NCP_D= self.load_data(self.path_list_NCP[randn])
        randn_aug_1=np.random.randint(0,3)
        if randn_aug_1==0:
            image_positive_NCP_D = fliplr(image_positive_NCP_D)
            mask_positive_NCP_D = fliplr(mask_positive_NCP_D)
        if randn_aug_1==1:
            image_positive_NCP_D = transform.rotate(image_positive_NCP_D,2)
            mask_positive_NCP_D = transform.rotate(mask_positive_NCP_D,2)
        if randn_aug_1==2:
            image_positive_NCP_D = transform.rotate(image_positive_NCP_D,-2)
            mask_positive_NCP_D = transform.rotate(mask_positive_NCP_D,-2)
        image_positive_NCP_D,mask_positive_NCP_D = self.normalize_data(image_positive_NCP_D,mask_positive_NCP_D)
        positive_map_D = scipy.ndimage.distance_transform_edt(mask_positive_NCP_D)
        if positive_map_D.max()>0:
            positive_map_D = positive_map_D/positive_map_D.max()
        ###### negative
        randn_1 = np.random.randint(0,num_CP_Normal)
        image_negative_NCP_D,mask_negative_NCP_D= self.load_data(self.path_list_CP_Normal[randn_1])
        image_negative_NCP_D,mask_negative_NCP_D= self.normalize_data(image_negative_NCP_D,mask_negative_NCP_D)
        negative_map_D = scipy.ndimage.distance_transform_edt(mask_negative_NCP_D)
        if negative_map_D.max()>0:
            negative_map_D = negative_map_D/negative_map_D.max()
        
        
        ########## For similar case recommendation
        #### positive
        randn_aug_2 = np.random.randint(0,4)
        if randn_aug_2==0:
            image_positive_NCP_R = fliplr(image_anchor_NCP)
            mask_positive_NCP_R = fliplr(mask_anchor_NCP)
        elif randn_aug_0==1:
            image_positive_NCP_R = transform.rotate(image_anchor_NCP,2)
            mask_positive_NCP_R = transform.rotate(mask_anchor_NCP,2)
        else:
            image_positive_NCP_R = transform.rotate(image_anchor_NCP,-2)
            mask_positive_NCP_R = transform.rotate(mask_anchor_NCP,-2)
        image_positive_NCP_R,mask_positive_NCP_R = self.normalize_data(image_positive_NCP_R,mask_positive_NCP_R)
        positive_map_R = scipy.ndimage.distance_transform_edt(mask_positive_NCP_R)
        if positive_map_R.max()>0:
            positive_map_R = positive_map_R/positive_map_R.max()
        
        ###### negative
        image_negative_NCP_R=np.zeros(256)
        mask_negative_NCP_R=np.zeros(256)
        original_patient = self.path_list_NCP[index].split('/')[-1].split('_')[1]
        for i in range(1000):
            randn_2 = np.random.randint(0,num_NCP)
            new_patient = self.path_list_NCP[randn_2].split('/')[-1].split('_')[1]
            if original_patient == new_patient:
                continue
            else:
                image_negative_NCP_R,mask_negative_NCP_R = self.load_data(self.path_list_NCP[randn_2])
                break
        randn_aug_3 = np.random.randint(0,4)
        if randn_aug_3==0:
            image_negative_NCP_R = fliplr(image_negative_NCP_R)
            mask_negative_NCP_R = fliplr(mask_negative_NCP_R)
        if randn_aug_3==1:
            image_negative_NCP_R = transform.rotate(image_negative_NCP_R,2)
            mask_negative_NCP_R = transform.rotate(mask_negative_NCP_R,2)
        if randn_aug_3==2:
            image_negative_NCP_R = transform.rotate(image_negative_NCP_R,-2)
            mask_negative_NCP_R = transform.rotate(mask_negative_NCP_R,-2)
        image_negative_NCP_R,mask_negative_NCP_R = self.normalize_data(image_negative_NCP_R,mask_negative_NCP_R)
        negative_map_R = scipy.ndimage.distance_transform_edt(mask_negative_NCP_R)
        if negative_map_R.max()>0:
            negative_map_R = negative_map_R/negative_map_R.max()
        label_anchor = 0
        label_positive = 0
        label_negative = 1
        return self.process_data(image_anchor_NCP,mask_anchor_NCP,anchor_map, \
                image_positive_NCP_D,mask_positive_NCP_D,positive_map_D,  image_negative_NCP_D,mask_negative_NCP_D,negative_map_D, \
                image_positive_NCP_R,mask_positive_NCP_R,positive_map_R,  image_negative_NCP_R,mask_negative_NCP_R,negative_map_R, \
                                label_anchor,label_positive,label_negative)
        
    def load_data(self, file_path):
        image = cv2.imread(file_path,0)
        image_type = file_path.split('/')[-2].split('_')[0]
        if image_type == 'NCP':
            mask_path = os.path.join('/data2/Cpeng/NCP_lung_masks/',file_path.split('/')[-1])
        elif image_type == 'CP':
            mask_path = os.path.join('/data2/Cpeng/CP_lung_masks/',file_path.split('/')[-1])
        else:
            mask_path = os.path.join('/data2/Cpeng/Normal_lung_masks/',file_path.split('/')[-1])
        mask = cv2.imread(mask_path,0)
        
        return image, mask
    
    
    def normalize_data(self,image,mask):
        image = cv2.resize(image,(224,224), interpolation=cv2.INTER_CUBIC)
        image = image-image.min()
        image = (image/image.max()).astype(np.float32)
        mask = cv2.resize(mask,(224,224), interpolation=cv2.INTER_CUBIC)
        mask[mask>0]=1.
        return image, mask
        
    def process_data(self, *args):
        a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15 = [item[np.newaxis, ...].astype(np.float32) for item in args[0:15]]
        a16,a17,a18 = [item for item in args[15:18]]
        return a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18
        
    def __len__(self):
        return len(self.path_list_NCP)

In [7]:
import torchvision.models as models
from config import *
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features,128)
coarse_network = nn.Sequential(*list(model.children())[0:5])
fine_network = nn.Sequential(*list(model.children())[5:9])
class COVID_CR_network(nn.Module):
    def __init__(self):
        super(COVID_CR_network,self).__init__()
        self.main_network = nn.Sequential(*list(model.children())[0:5])
        self.Detection_network = nn.Sequential(*list(model.children())[5:9])
        self.Recommendation_network = nn.Sequential(*list(model.children())[5:9])
        self.fc1 = nn.Linear(512,128)
        self.fc2 = nn.Linear(512,128)
        self.fc3 = nn.Linear(128, 2)
#     def l2_norm(self,input):
#         input_size = input.size()
#         buffer = torch.pow(input, 2)

#         normp = torch.sum(buffer, 1).add_(1e-10)
#         norm = torch.sqrt(normp)

#         _output = torch.div(input, norm.view(-1, 1).expand_as(input))

#         output = _output.view(input_size)

#         return output
    def forward(self,image_anchor,mask_anchor,map_anchor,
                image_positive_D,mask_positive_D,map_positive_D,image_negative_D,mask_negative_D,map_negative_D,
                image_positive_R,mask_positive_R,map_positive_R,image_negative_R,mask_negative_R,map_negative_R):
        
        input_anchor = torch.cat([image_anchor,image_anchor*mask_anchor,map_anchor],dim=1)
        
        input_positive_D = torch.cat([image_positive_D,image_positive_D*mask_positive_D,map_positive_D],dim=1)
        input_negative_D = torch.cat([image_negative_D,image_negative_D*mask_negative_D,map_negative_D],dim=1)
        output_anchor_D = self.Detection_network(self.main_network(input_anchor))
        output_positive_D = self.Detection_network(self.main_network(input_positive_D))
        output_negative_D = self.Detection_network(self.main_network(input_negative_D))
        
        output_anchor_D = output_anchor_D.view(output_anchor_D.size(0), -1)
        output_positive_D = output_positive_D.view(output_positive_D.size(0), -1)
        output_negative_D = output_negative_D.view(output_negative_D.size(0), -1)
        output_anchor_D = self.fc1(output_anchor_D)
        output_positive_D = self.fc1(output_positive_D)
        output_negative_D = self.fc1(output_negative_D)
        
        
        input_positive_R = torch.cat([image_positive_R,image_positive_R*mask_positive_R,map_positive_R],dim=1)
        input_negative_R = torch.cat([image_negative_R,image_negative_R*mask_negative_R,map_negative_R],dim=1)
        output_anchor_R = self.Recommendation_network(self.main_network(input_anchor))
        output_positive_R = self.Recommendation_network(self.main_network(input_positive_R))
        output_negative_R = self.Recommendation_network(self.main_network(input_negative_R))
        
        
        output_anchor_R = output_anchor_R.view(output_anchor_R.size(0), -1)
        output_positive_R = output_positive_R.view(output_positive_R.size(0), -1)
        output_negative_R = output_negative_R.view(output_negative_R.size(0), -1)
        
        output_anchor_R = self.fc2(output_anchor_R)
        output_positive_R = self.fc2(output_positive_R)
        output_negative_R = self.fc2(output_negative_R)
        
        return output_anchor_D,output_positive_D,output_negative_D,output_anchor_R,output_positive_R,output_negative_R
    def classification(self,input_features):
        output = self.fc3(input_features)
        return output

In [8]:
device_ids = [0,1,2,3]
COVID_DR_model = COVID_CR_network()
# checkpoint = torch.load('/data2/Cpeng/model_COVID_DR/checkpoint_120.pth.tar', map_location="cuda:0")
# COVID_DR_model.load_state_dict(checkpoint['state_dict'])
COVID_DR_model.cuda(device=device_ids[0])
COVID_DR_model = nn.DataParallel(COVID_DR_model,device_ids = device_ids)
print(COVID_DR_model)

DataParallel(
  (module): COVID_CR_network(
    (main_network): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNo

In [9]:
torch.backends.cudnn.benchmark = True
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')
batch_size = 64*4
num_workers = 8

In [10]:
trainset = COVID_19(NCP_train,CP_Normal_train, phase='train')
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

valset = COVID_19(NCP_val,CP_Normal_val, phase='train')
val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [11]:
from torch.optim.lr_scheduler import *
criterion_1 = nn.TripletMarginLoss(margin=1.2).cuda()
criterion_2 = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(COVID_DR_model.parameters(), lr=1e-3,weight_decay=1e-5)
optimizer = nn.DataParallel(optimizer, device_ids=device_ids)
scheduler = StepLR(optimizer.module, 20, gamma=0.5)

In [12]:
def save_checkpoint(state, checkpoint='checkpoint', snapshot=1):
    if not os.path.exists(checkpoint): os.makedirs(checkpoint)
    if snapshot and state['epoch'] % snapshot == 0:
        torch.save(state, os.path.join(checkpoint, 'checkpoint_{}.pth.tar'.format(state['epoch'])))

In [3]:
step=0
epoches=401
margin =1.2
############
l2_dist = PairwiseDistance(2)
def l2_norm(input):
        input_size = input.size()
        buffer = torch.pow(input, 2)
        
        normp = torch.sum(buffer, 1).add_(1e-10)
        norm = torch.sqrt(normp)
        
        _output = torch.div(input, norm.view(-1, 1).expand_as(input))
        
        output = _output.view(input_size)
        
        return output


loss_train=0.0
loss_val=0.0
triplet_loss_train_D = 0.0
triplet_loss_train_R = 0.0
triplet_loss_val_D = 0.0
triplet_loss_val_R = 0.0
CE_loss_train_D = 0.0
CE_loss_val_D = 0.0
output_anchor_D = torch.linspace(-1,1,128)
output_positive_D = torch.linspace(-1,1,128)
output_negative_D = torch.linspace(-1,1,128)
output_anchor_R = torch.linspace(-1,1,128)
output_positive_R = torch.linspace(-1,1,128)
output_negative_R = torch.linspace(-1,1,128)
for epoch in range(1,epoches):
    print('epoch:' + str(epoch))
    
    COVID_DR_model.train()
    for i,data in enumerate(train_loader):
        image_anchor,mask_anchor,map_anchor,image_positive_D,mask_positive_D,map_positive_D,image_negative_D,mask_negative_D,map_negative_D, \
                image_positive_R,mask_positive_R,map_positive_R,image_negative_R,mask_negative_R,map_negative_R, \
                                label_anchor,label_positive,label_negative = [item.to(device) for item in data]
        
        output_anchor_D,output_positive_D,output_negative_D,output_anchor_R,output_positive_R,output_negative_R = \
                        COVID_DR_model(image_anchor,mask_anchor,map_anchor,
                                       image_positive_D,mask_positive_D,map_positive_D,image_negative_D,mask_negative_D,map_negative_D,
                                       image_positive_R,mask_positive_R,map_positive_R,image_negative_R,mask_negative_R,map_negative_R)
        
        
        output_anchor_D = l2_norm(output_anchor_D)
        output_positive_D = l2_norm(output_positive_D)
        output_negative_D = l2_norm(output_negative_D)
        output_anchor_R = l2_norm(output_anchor_R)
        output_positive_R = l2_norm(output_positive_R)
        output_negative_R = l2_norm(output_negative_R)
        
        # Choose the hard negatives
#         d_p_D = l2_dist.forward(output_anchor_D, output_positive_D)
#         d_n_D = l2_dist.forward(output_anchor_D, output_negative_D)
#         all_D = (d_n_D - d_p_D < margin).cpu().data.numpy().flatten()
#         hard_triplets_D = np.where(all_D == 1)
#         if len(hard_triplets_D[0]) == 0:
#             continue
#         out_selected_a_D = torch.from_numpy(output_anchor_D.cpu().data.numpy()[hard_triplets_D]).cuda()
#         out_selected_p_D = torch.from_numpy(output_positive_D.cpu().data.numpy()[hard_triplets_D]).cuda()
#         out_selected_n_D = torch.from_numpy(output_negative_D.cpu().data.numpy()[hard_triplets_D]).cuda()
        # select data
#         selected_image_a_D = torch.from_numpy(image_anchor.cpu().data.numpy()[hard_triplets_D]).cuda()
#         selected_image_p_D = torch.from_numpy(image_positive_D.cpu().data.numpy()[hard_triplets_D]).cuda()
#         selected_image_n_D = torch.from_numpy(image_negative_D.cpu().data.numpy()[hard_triplets_D]).cuda()
        
#         selected_mask_a_D = torch.from_numpy(mask_anchor.cpu().data.numpy()[hard_triplets_D]).cuda()
#         selected_mask_p_D = torch.from_numpy(mask_positive_D.cpu().data.numpy()[hard_triplets_D]).cuda()
#         selected_mask_n_D = torch.from_numpy(mask_negative_D.cpu().data.numpy()[hard_triplets_D]).cuda()
        
#         selected_map_a_D = torch.from_numpy(map_anchor.cpu().data.numpy()[hard_triplets_D]).cuda()
#         selected_map_p_D = torch.from_numpy(map_positive_D.cpu().data.numpy()[hard_triplets_D]).cuda()
#         selected_map_n_D = torch.from_numpy(map_negative_D.cpu().data.numpy()[hard_triplets_D]).cuda()
        
#         selected_label_p = torch.from_numpy(label_positive.cpu().numpy()[hard_triplets_D])
#         selected_label_n= torch.from_numpy(label_negative.cpu().numpy()[hard_triplets_D])
#         print(selected_image_a_D.size())
#         print(selected_image_a_D.max())
        ### classification
#         print(selected_image_a_D.size())
#         print(selected_image_p_D.size())
#         output_anchor_D_1,output_positive_D_1,output_negative_D_1 = COVID_DR_model(
#                     selected_image_a_D,selected_mask_a_D,selected_map_a_D,
#                     selected_image_p_D,selected_mask_p_D,selected_map_p_D,selected_image_n_D,selected_mask_n_D,selected_map_n_D,
#                     image_positive_R,mask_positive_R,map_positive_R,image_negative_R,mask_negative_R,map_negative_R)
        
        #compute loss
#         triplet_loss_train_D = criterion_1(out_selected_a_D, out_selected_p_D, out_selected_n_D)
        triplet_loss_train_D = criterion_1(output_anchor_D,output_positive_D,output_negative_D)
        
        
#         predicted_labels = torch.cat([output_anchor_D_1,output_positive_D_1,output_negative_D_1])
#         true_labels = torch.cat([selected_label_p.cuda(),selected_label_p.cuda(),selected_label_n.cuda()])
#         print(output_anchor_D_1.size())
#         print(label_anchor.size())
        output_anchor_D_1 = COVID_DR_model.module.classification(output_anchor_D)
        output_positive_D_1 = COVID_DR_model.module.classification(output_positive_D)
        output_negative_D_1 = COVID_DR_model.module.classification(output_negative_D)
        CE_loss_train_D = criterion_2(output_anchor_D_1,label_anchor)+ \
                        criterion_2(output_positive_D_1,label_positive)+ \
                        criterion_2(output_negative_D_1,label_negative)
#         CE_loss_val_D = criterion_2(output_anchor_D_1.view(-1, 1),label_anchor.view(-1, 1))+ \
#                             criterion_2(output_positive_D_1.view(-1, 1),label_positive.view(-1, 1))+ \
#                             criterion_2(output_negative_D_1.view(-1, 1),label_negative.view(-1, 1))
        ############ for Retrival
        # Choose the hard negatives
#         d_p_R = l2_dist.forward(output_anchor_R, output_positive_R)
#         d_n_R = l2_dist.forward(output_anchor_R, output_negative_R)
#         all_R = (d_n_R - d_p_R < margin).cpu().data.numpy().flatten()
#         hard_triplets_R = np.where(all_R == 1)
#         if len(hard_triplets_R[0]) == 0:
#             continue
#         out_selected_a_R = Variable(torch.from_numpy(output_anchor_R.cpu().data.numpy()[hard_triplets_R]).cuda())
#         out_selected_p_R = Variable(torch.from_numpy(output_positive_R.cpu().data.numpy()[hard_triplets_R]).cuda())
#         out_selected_n_R = Variable(torch.from_numpy(output_negative_R.cpu().data.numpy()[hard_triplets_R]).cuda())
        #compute loss
        triplet_loss_train_R = criterion_1(output_anchor_R,output_positive_R,output_negative_R)
        
        
        
#         loss_train_D=triplet_loss_train_D + CE_loss_train_D
#         loss_train_D=triplet_loss_train_D
#         loss_train_R=triplet_loss_train_R
        loss_train = 0.3*(0.4*triplet_loss_train_D+0.6*CE_loss_train_D)+0.7*triplet_loss_train_R
        optimizer.module.zero_grad()
        loss_train.backward()
        optimizer.module.step()
    with torch.no_grad():
        COVID_DR_model.eval()
        for i,data in enumerate(val_loader):
            image_anchor,mask_anchor,map_anchor,image_positive_D,mask_positive_D,map_positive_D,image_negative_D,mask_negative_D,map_negative_D, \
                image_positive_R,mask_positive_R,map_positive_R,image_negative_R,mask_negative_R,map_negative_R, \
                                label_anchor,label_positive,label_negative = [item.to(device) for item in data]
            output_anchor_D,output_positive_D,output_negative_D,output_anchor_R,output_positive_R,output_negative_R = \
                        COVID_DR_model(image_anchor,mask_anchor,map_anchor,
                                       image_positive_D,mask_positive_D,map_positive_D,image_negative_D,mask_negative_D,map_negative_D,
                                       image_positive_R,mask_positive_R,map_positive_R,image_negative_R,mask_negative_R,map_negative_R)
            
            output_anchor_D = l2_norm(output_anchor_D)
            output_positive_D = l2_norm(output_positive_D)
            output_negative_D = l2_norm(output_negative_D)
            output_anchor_R = l2_norm(output_anchor_R)
            output_positive_R = l2_norm(output_positive_R)
            output_negative_R = l2_norm(output_negative_R)
            
            # Choose the hard negatives
#             d_p_D = l2_dist.forward(output_anchor_D, output_positive_D)
#             d_n_D = l2_dist.forward(output_anchor_D, output_negative_D)
#             all_D = (d_n_D - d_p_D < margin).cpu().data.numpy().flatten()
#             hard_triplets_D = np.where(all_D == 1)
#             if len(hard_triplets_D[0]) == 0:
#                 continue
#             out_selected_a_D = Variable(torch.from_numpy(output_anchor_D.cpu().data.numpy()[hard_triplets_D]).cuda())
#             out_selected_p_D = Variable(torch.from_numpy(output_positive_D.cpu().data.numpy()[hard_triplets_D]).cuda())
#             out_selected_n_D = Variable(torch.from_numpy(output_negative_D.cpu().data.numpy()[hard_triplets_D]).cuda())
            # select data
#             selected_image_a_D = Variable(torch.from_numpy(image_anchor.cpu().data.numpy()[hard_triplets_D]).cuda())
#             selected_image_p_D = Variable(torch.from_numpy(image_positive_D.cpu().data.numpy()[hard_triplets_D]).cuda())
#             selected_image_n_D = Variable(torch.from_numpy(image_negative_D.cpu().data.numpy()[hard_triplets_D]).cuda())

#             selected_mask_a_D = Variable(torch.from_numpy(mask_anchor.cpu().data.numpy()[hard_triplets_D]).cuda())
#             selected_mask_p_D = Variable(torch.from_numpy(mask_positive_D.cpu().data.numpy()[hard_triplets_D]).cuda())
#             selected_mask_n_D = Variable(torch.from_numpy(mask_negative_D.cpu().data.numpy()[hard_triplets_D]).cuda())

#             selected_map_a_D = Variable(torch.from_numpy(map_anchor.cpu().data.numpy()[hard_triplets_D]).cuda())
#             selected_map_p_D = Variable(torch.from_numpy(map_positive_D.cpu().data.numpy()[hard_triplets_D]).cuda())
#             selected_map_n_D = Variable(torch.from_numpy(map_negative_D.cpu().data.numpy()[hard_triplets_D]).cuda())

#             selected_label_p = torch.from_numpy(label_positive.cpu().numpy()[hard_triplets_D])
#             selected_label_n= torch.from_numpy(label_negative.cpu().numpy()[hard_triplets_D])

#             ### classification
#             output_anchor_D_1,output_positive_D_1,output_negative_D_1 = COVID_DR_model(
#                     selected_image_a_D,selected_mask_a_D,selected_map_a_D,
#                     selected_image_p_D,selected_mask_p_D,selected_map_p_D,selected_image_n_D,selected_mask_n_D,selected_map_n_D,
#                     image_positive_R,mask_positive_R,map_positive_R,image_negative_R,mask_negative_R,map_negative_R)
            
            #compute loss
            triplet_loss_val_D = criterion_1(output_anchor_D,output_positive_D,output_negative_D)
            
#             predicted_labels = torch.cat([output_anchor_D_1,output_positive_D_1,output_negative_D_1])
#             true_labels = torch.cat([selected_label_p.cuda(),selected_label_p.cuda(),selected_label_n.cuda()])
            output_anchor_D_1 = COVID_DR_model.module.classification(output_anchor_D)
            output_positive_D_1 = COVID_DR_model.module.classification(output_positive_D)
            output_negative_D_1 = COVID_DR_model.module.classification(output_negative_D)
            CE_loss_val_D = criterion_2(output_anchor_D_1,label_anchor)+ \
                        criterion_2(output_positive_D_1,label_positive)+ \
                        criterion_2(output_negative_D_1,label_negative)
#             CE_loss_val_D = criterion_2(output_anchor_D_1.view(-1, 1),label_anchor.view(-1, 1))+ \
#                             criterion_2(output_positive_D_1.view(-1, 1),label_positive.view(-1, 1))+ \
#                             criterion_2(output_negative_D_1.view(-1, 1),label_negative.view(-1, 1))
#             CE_loss_val_D = criterion_2(predicted_labels.cuda(),true_labels.cuda())
            ############ for Retrival
            # Choose the hard negatives
#             d_p_R = l2_dist.forward(output_anchor_R, output_positive_R)
#             d_n_R = l2_dist.forward(output_anchor_R, output_negative_R)
#             all_R = (d_n_R - d_p_R < margin).cpu().data.numpy().flatten()
#             hard_triplets_R = np.where(all_R == 1)
#             if len(hard_triplets_R[0]) == 0:
#                 continue
#             out_selected_a_R = Variable(torch.from_numpy(output_anchor_R.cpu().data.numpy()[hard_triplets_R]).cuda())
#             out_selected_p_R = Variable(torch.from_numpy(output_positive_R.cpu().data.numpy()[hard_triplets_R]).cuda())
#             out_selected_n_R = Variable(torch.from_numpy(output_negative_R.cpu().data.numpy()[hard_triplets_R]).cuda())
            #compute loss
            triplet_loss_val_R = criterion_1(output_anchor_R,output_positive_R,output_negative_R)
            
            
            
#             loss_val_D=triplet_loss_val_D + CE_loss_val_D
#             loss_val_D=triplet_loss_val_D
#             loss_val_R=triplet_loss_val_R
            loss_val = 0.3*(0.4*triplet_loss_val_D+0.6*CE_loss_val_D)+0.7*triplet_loss_val_R
    scheduler.step()
#     print('Triplet_Loss_Detection_train: {}'.format(triplet_loss_train_D))
#     print('CE_Loss_train: {}'.format(CE_loss_train_D))
    print('Train_Detection_CE_Loss: {}'.format(CE_loss_train_D))
    print('Train_Detection_Loss: {}'.format(triplet_loss_train_D))
    print('#########')
    print('Train_Recommendation_Loss: {}'.format(triplet_loss_train_R))
    print('Train_Loss: {}'.format(loss_train))
    print('#####################')
#     print('Triplet_Loss_Detection_val: {}'.format(triplet_loss_val_D))
    print('Val_Detection_CE_Loss: {}'.format(CE_loss_val_D))
    print('Val_Detection_Loss: {}'.format(triplet_loss_val_D))
    print('#########')
    print('Val_Recommendation_Loss: {}'.format(triplet_loss_val_R))
    print('Val_Loss: {}'.format(loss_val))
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
#     writer.add_scalar('train_loss',loss_train, epoch)
#     writer.add_scalar('val_loss',loss_val, epoch)
    if epoch%10==0:
        save_checkpoint({
            'epoch': epoch,
            'state_dict': COVID_DR_model.module.state_dict(),
            }, checkpoint='/data2/Cpeng/model_COVID_DR', snapshot=1)

In [1]:
index=24
path = NCP_train[index]
test_image = cv2.imread(path,0)
test_image = cv2.resize(test_image,(224,224), interpolation=cv2.INTER_CUBIC)
test_image = test_image-test_image.min()
test_image = (test_image/test_image.max()).astype(np.float32)
plt.imshow(test_image)
test_image_ = torch.from_numpy(test_image[np.newaxis,np.newaxis,...]).cuda()
print(type(test_image_))
image_type = path.split('/')[-2].split('_')[0]
print(image_type)
if image_type == 'NCP':
    mask_path = os.path.join('/data2/Cpeng/NCP_lung_masks/',path.split('/')[-1])
elif image_type == 'CP':
    mask_path = os.path.join('/data2/Cpeng/CP_lung_masks/',path.split('/')[-1])
else:
    mask_path = os.path.join('/data2/Cpeng/Normal_lung_masks/',path.split('/')[-1])
mask = cv2.imread(mask_path,0)
mask = cv2.resize(mask,(224,224), interpolation=cv2.INTER_CUBIC)
mask[mask>0]=1.
mask = mask.astype(np.float32)
plt.figure()
plt.imshow(mask)
mask_map = scipy.ndimage.distance_transform_edt(mask)
mask_map = (mask_map/mask_map.max()).astype(np.float32)
plt.figure()
plt.imshow(mask_map)
mask_ = torch.from_numpy(mask[np.newaxis,np.newaxis,...]).cuda()
mask_map_ = torch.from_numpy(mask_map[np.newaxis,np.newaxis,...]).cuda()

In [59]:
COVID_DR_model = COVID_CR_network()
checkpoint = torch.load('/data2/Cpeng/model_COVID_DR/checkpoint_30.pth.tar', map_location="cuda:0")
COVID_DR_model.load_state_dict(checkpoint['state_dict'])
COVID_DR_model.cuda()
output_anchor_D,output_positive_D,output_negative_D,output_anchor_R,output_positive_R,output_negative_R = \
        COVID_DR_model(test_image_,mask_,mask_map_,
                   test_image_,mask_,mask_map_,
                   test_image_,mask_,mask_map_,
                   test_image_,mask_,mask_map_,
                   test_image_,mask_,mask_map_)
output_anchor_D_1 =COVID_DR_model.classification(output_anchor_D) 
print(torch.softmax(output_anchor_D_1,dim=1))
print(output_anchor_D)

tensor([[0.9845, 0.0155]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[ 0.4030,  0.1717, -0.2607,  0.0182,  0.5311, -0.8104, -0.0843,  0.3068,
         -0.7838, -0.2688, -0.1498,  0.3320,  0.1315,  0.4377,  0.3430,  1.0309,
         -0.7670,  0.7076,  0.0818,  0.2519,  0.3087, -0.3093, -0.7321,  0.9076,
          0.4248,  0.1441,  0.4799, -0.4326, -0.3921,  0.9561,  0.2539, -0.3004,
         -0.2766, -0.4824, -0.3906, -0.0180, -0.4645,  0.9989,  0.2780, -0.2343,
         -0.2929,  0.2809, -0.0876, -0.3067, -0.2799, -0.3968, -0.2292, -0.3067,
          0.0908, -0.4303,  0.7445,  0.1691, -0.0073,  0.3704,  0.3450, -0.2669,
          0.1401, -0.3687,  0.6973,  0.3545, -0.3329,  0.2113,  0.2049,  0.1024,
         -0.5781, -0.5623, -0.1788, -0.1551, -0.0813, -0.0860,  0.1842, -0.1160,
         -0.2571,  0.2024, -0.3429, -0.1812, -0.0159, -0.3434,  0.2976, -0.0879,
          0.6142,  0.3036, -0.2015, -0.3351, -0.2978, -1.1246, -0.1829,  0.3965,
         -0.6307, -0.1248, -0.3157, -0